In [2]:
# Classification
import math
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import GlobalAveragePooling1D, Conv1D, GlobalMaxPooling1D, BatchNormalization
from keras.layers import Dense
from keras.regularizers import L1L2
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses, regularizers
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, confusion_matrix, auc
from sklearn.metrics import roc_auc_score, precision_recall_curve
import pandas as pd
import time
import os

In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold, cross_val_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf

def normalize(data):
    min = np.min(data, axis=0)
    max = np.max(data, axis=0)
    data = (data - min) / (max - min)
    return data

def create_model(filter_sz=64, kernel_sz1=3, kernel_sz2=3, dropout=0.4, input_shape=[None, 1]):
    model = Sequential()
    model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz1, strides=1, activation='relu', input_shape=input_shape, name='L1'))
    model.add(Dropout(dropout))  # dropout rate of 0.4
    model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz2, strides=1, activation='relu', name='L2'))
    model.add(Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', name='L3'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropout))  # dropout rate of 0.4 or 0.6
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid', name='classification'))
    adam = tf.keras.optimizers.Adam(learning_rate=1.e-04)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

# Fix random seed for reproducibility
seed = 128
np.random.seed(seed)

# Prepare an Excel writer object
file_path = "RFE_performance.xlsx"
writer = pd.ExcelWriter(file_path, engine='openpyxl')

# Loop over 13 subtasks
for st_num in range(1):  # Change to `range(13)` to loop over all 13 subtasks
    dir = r'C:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\EEG_fNIRS_paper_Brain_informatics\channelEEG_codes_results_alphaBand\Results\Connectivities_LSTMED_multiChanROI_Copy'
    filename = 'exp_nov_winSiz_5ST' + str(st_num + 1) + '_Copy.csv'
    print(f'working on : {filename}')

    data = pd.read_csv(os.path.join(dir, filename)).values
    m, n = data.shape
    X0 = normalize(data[:, 0:n-1])
    X = X0.reshape(X0.shape[0], X0.shape[1], 1)
    X = normalize(X)
    y = data[:, n-1]
    y = np.array([0 if y[i] == -1 else 1 for i in range(len(y))])

    # Initialize RFE with cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Range of number of features to consider
    min_features_to_select = 1
    rfe = RFE(estimator=KerasClassifier(build_fn=create_model, epochs=10, batch_size=16, verbose=0), n_features_to_select=min_features_to_select, step=1)

    # Run RFE with cross-validation
    mean_scores = []
    feature_counts = list(range(min_features_to_select, X.shape[1] + 1))

    for i in feature_counts:
        rfe.n_features_to_select = i
        scores = cross_val_score(rfe, X, y, cv=cv)
        mean_scores.append(scores.mean())

        # Create a DataFrame to store results for the current number of features
        result_df = pd.DataFrame({
            'Fold': range(1, cv.get_n_splits(X, y) + 1),
            'Accuracy': scores
        })
        result_df.loc['Mean'] = result_df.mean()  # Add mean row

        # Write results to a separate sheet
        sheet_name = f'ST{st_num + 1}_Features_{i}'
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Combine the results into a summary DataFrame and sort by accuracy
    summary_df = pd.DataFrame({
        'Number of Features': feature_counts,
        'Mean Accuracy': mean_scores
    })
    summary_df = summary_df.sort_values(by='Mean Accuracy')

    # Write the summary DataFrame to a new sheet
    summary_sheet_name = f'ST{st_num + 1}_Summary'
    summary_df.to_excel(writer, sheet_name=summary_sheet_name, index=False)

    # Plot the cross-validated scores for the current subtask
    plt.figure(figsize=(10, 6))
    plt.plot(feature_counts, mean_scores, marker='o')
    plt.xlabel('Number of Features')
    plt.ylabel('Cross-Validated Accuracy')
    plt.title(f'RFE with Cross-Validation for Subtask {st_num + 1}')
    plt.grid()
    plt.show()

    # Find the optimal number of features
    optimal_num_features = feature_counts[mean_scores.index(max(mean_scores))]
    print(f"Optimal number of features for Subtask {st_num + 1}: {optimal_num_features}")

# Save the Excel file
writer.save()


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [42]:
import random
import math

class SimpleLinearRegression:
    def __init__(self):
        self.coefficients = None
        self.intercept = None

    def fit(self, X, y):
        n = len(X)
        x_mean = sum(X) / n
        y_mean = sum(y) / n
        
        numerator = sum((X[i] - x_mean) * (y[i] - y_mean) for i in range(n))
        denominator = sum((X[i] - x_mean) ** 2 for i in range(n))
        
        self.coefficients = numerator / denominator
        self.intercept = y_mean - self.coefficients * x_mean

    def predict(self, X):
        return [self.coefficients * x + self.intercept for x in X]

def mean_squared_error(y_true, y_pred):
    return sum((y_true[i] - y_pred[i]) ** 2 for i in range(len(y_true))) / len(y_true)

def recursive_feature_elimination(X, y, n_features_to_select):
    n_features = len(X[0])
    feature_indices = list(range(n_features))
    selected_features = []
    
    while len(feature_indices) > n_features_to_select:
        best_mse = float('inf')
        worst_feature = None
        
        for i in feature_indices:
            current_features = [f for f in feature_indices if f != i]
            X_subset = [[row[j] for j in current_features] for row in X]
            
            mse_sum = 0
            for _ in range(5):  # 5-fold cross-validation
                train_indices = random.sample(range(len(X)), int(0.8 * len(X)))
                test_indices = [i for i in range(len(X)) if i not in train_indices]
                
                X_train = [X_subset[i] for i in train_indices]
                y_train = [y[i] for i in train_indices]
                X_test = [X_subset[i] for i in test_indices]
                y_test = [y[i] for i in test_indices]
                
                model = SimpleLinearRegression()
                model.fit([row[0] for row in X_train], y_train)
                y_pred = model.predict([row[0] for row in X_test])
                mse = mean_squared_error(y_test, y_pred)
                mse_sum += mse
            
            avg_mse = mse_sum / 5
            
            if avg_mse < best_mse:
                best_mse = avg_mse
                worst_feature = i
        
        feature_indices.remove(worst_feature)
        selected_features.append(worst_feature)
    
    return feature_indices[::-1] + selected_features[::-1]

# Example usage
X = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]
y = [2, 8, 7, 3]
n_features_to_select = 2

selected_features = recursive_feature_elimination(X, y, n_features_to_select)
print("Selected features (in order of importance):", selected_features)

Selected features (in order of importance): [2, 1, 0, 3]


In [43]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, Dropout, GlobalAveragePooling1D, Dense
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# def create_model(filter_sz=64, kernel_sz1=3, kernel_sz2=3, dropout=0.4, input_shape=[None, 1]):
#     model = Sequential()
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz1, strides=1, activation='relu', input_shape=input_shape, name='L1'))
#     model.add(Dropout(dropout))
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz2, strides=1, activation='relu', name='L2'))
#     model.add(Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', name='L3'))
#     model.add(GlobalAveragePooling1D())
#     model.add(Dropout(dropout))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1, activation='sigmoid', name='classification'))
#     adam = tf.keras.optimizers.Adam(learning_rate=1.e-04)
#     model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
#     return model

# def recursive_feature_elimination(X, y, n_features_to_select):
#     n_features = X.shape[1]
#     feature_indices = list(range(n_features))
#     selected_features = []
    
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     while len(feature_indices) > n_features_to_select:
#         best_accuracy = 0
#         worst_feature = None
        
#         for i in feature_indices:
#             current_features = [f for f in feature_indices if f != i]
#             X_subset = X_scaled[:, current_features]
            
#             X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.2, random_state=42)
            
#             X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#             X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
            
#             model = create_model(input_shape=[X_train.shape[1], 1])
#             model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
            
#             _, accuracy = model.evaluate(X_test, y_test, verbose=0)
            
#             if accuracy > best_accuracy:
#                 best_accuracy = accuracy
#                 worst_feature = i
        
#         feature_indices.remove(worst_feature)
#         selected_features.append(worst_feature)
        
#         print(f"Removed feature {worst_feature}, {len(feature_indices)} features remaining")
    
#     return feature_indices[::-1] + selected_features[::-1]

# # Generate example data
# np.random.seed(42)
# X = np.random.rand(100, 20)
# y = (X[:, 0] + X[:, 1] + X[:, 2] > 1.5).astype(int)  # Binary classification based on first 3 features

# n_features_to_select = 5

# selected_features = recursive_feature_elimination(X, y, n_features_to_select)
# print("Selected features (in order of importance):", selected_features)

c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Removed feature 5, 19 features remaining
Removed feature 18, 18 features remaining
Removed feature 15, 17 features remaining
Removed feature 14, 16 features remaining
Removed feature 13, 15 features remaining
Removed feature 16, 14 features remaining
Removed feature 9, 13 features remaining
Removed feature 6, 12 features remaining
Removed feature 3, 11 features remaining
Removed feature 12, 10 features remaining
Removed feature 10, 9 features remaining
Removed feature 8, 8 features remaining
Removed feature 4, 7 features remaining
Removed feature 11, 6 features remaining
Removed feature 7, 5 features remaining
Selected features (in order of importance): [19, 17, 2, 1, 0, 7, 11, 4, 8, 10, 12, 3, 6, 9, 16, 13, 14, 15, 18, 5]


In [45]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, Dropout, GlobalAveragePooling1D, Dense
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler

# def create_model(filter_sz=64, kernel_sz1=3, kernel_sz2=3, dropout=0.4, input_shape=[None, 1]):
#     model = Sequential()
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz1, strides=1, activation='relu', input_shape=input_shape, name='L1'))
#     model.add(Dropout(dropout))
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz2, strides=1, activation='relu', name='L2'))
#     model.add(Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', name='L3'))
#     model.add(GlobalAveragePooling1D())
#     model.add(Dropout(dropout))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1, activation='sigmoid', name='classification'))
#     adam = tf.keras.optimizers.Adam(learning_rate=1.e-04)
#     model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
#     return model

# def cross_val_accuracy(X, y):
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     accuracies = []
    
#     for train_index, val_index in kf.split(X):
#         X_train, X_val = X[train_index], X[val_index]
#         y_train, y_val = y[train_index], y[val_index]
        
#         X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#         X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
        
#         model = create_model(input_shape=[X_train.shape[1], 1])
#         model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        
#         _, accuracy = model.evaluate(X_val, y_val, verbose=0)
#         accuracies.append(accuracy)
    
#     return np.mean(accuracies)

# def recursive_feature_elimination(X, y):
#     n_features = X.shape[1]
#     feature_indices = list(range(n_features))
#     selected_features = []
    
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     best_accuracy = 0
#     best_feature_set = feature_indices.copy()
    
#     while len(feature_indices) > 1:
#         current_accuracy = cross_val_accuracy(X_scaled[:, feature_indices], y)
#         print(f"Current number of features: {len(feature_indices)}, Accuracy: {current_accuracy:.4f}")
        
#         if current_accuracy > best_accuracy:
#             best_accuracy = current_accuracy
#             best_feature_set = feature_indices.copy()
#         else:
#             print("Accuracy decreased. Stopping feature elimination.")
#             break
        
#         feature_importances = []
#         for i in feature_indices:
#             temp_features = [f for f in feature_indices if f != i]
#             accuracy = cross_val_accuracy(X_scaled[:, temp_features], y)
#             feature_importances.append((i, accuracy))
        
#         worst_feature, _ = max(feature_importances, key=lambda x: x[1])
#         feature_indices.remove(worst_feature)
#         selected_features.append(worst_feature)
        
#         print(f"Removed feature {worst_feature}")
    
#     return best_feature_set[::-1] + selected_features[::-1]

# # Generate example data
# np.random.seed(42)
# X = np.random.rand(100, 20)
# y = (X[:, 0] + X[:, 1] + X[:, 2] > 1.5).astype(int)  # Binary classification based on first 3 features

# selected_features = recursive_feature_elimination(X, y)
# print("Selected features (in order of importance):", selected_features)
# print(f"Optimal number of features: {len(selected_features)}")

c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Current number of features: 20, Accuracy: 0.5200
Removed feature 17
Current number of features: 19, Accuracy: 0.5200
Accuracy decreased. Stopping feature elimination.
Selected features (in order of importance): [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 17]
Optimal number of features: 21


In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, Dropout, GlobalAveragePooling1D, Dense
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler

# def create_model(filter_sz=64, kernel_sz1=3, kernel_sz2=3, dropout=0.4, input_shape=[None, 1]):
#     model = Sequential()
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz1, strides=1, activation='relu', input_shape=input_shape, name='L1'))
#     model.add(Dropout(dropout))
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz2, strides=1, activation='relu', name='L2'))
#     model.add(Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', name='L3'))
#     model.add(GlobalAveragePooling1D())
#     model.add(Dropout(dropout))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1, activation='sigmoid', name='classification'))
#     adam = tf.keras.optimizers.Adam(learning_rate=1.e-04)
#     model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
#     return model

# def cross_val_accuracy(X, y):
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     accuracies = []
    
#     for train_index, val_index in kf.split(X):
#         X_train, X_val = X[train_index], X[val_index]
#         y_train, y_val = y[train_index], y[val_index]
        
#         X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#         X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
        
#         model = create_model(input_shape=[X_train.shape[1], 1])
#         model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        
#         _, accuracy = model.evaluate(X_val, y_val, verbose=0)
#         accuracies.append(accuracy)
    
#     return np.mean(accuracies)

# def recursive_feature_elimination(X, y):
#     n_features = X.shape[1]
#     feature_indices = list(range(n_features))
#     selected_features = []
    
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     accuracies = []
#     feature_sets = []
    
#     while len(feature_indices) > 1:
#         current_accuracy = cross_val_accuracy(X_scaled[:, feature_indices], y)
#         accuracies.append(current_accuracy)
#         feature_sets.append(feature_indices.copy())
        
#         print(f"Current number of features: {len(feature_indices)}, Accuracy: {current_accuracy:.4f}")
        
#         feature_importances = []
#         for i in feature_indices:
#             temp_features = [f for f in feature_indices if f != i]
#             accuracy = cross_val_accuracy(X_scaled[:, temp_features], y)
#             feature_importances.append((i, accuracy))
        
#         worst_feature, _ = max(feature_importances, key=lambda x: x[1])
#         feature_indices.remove(worst_feature)
#         selected_features.append(worst_feature)
        
#         print(f"Removed feature {worst_feature}")
    
#     # Find the point where accuracy starts decreasing
#     best_accuracy_index = np.argmax(accuracies)
#     best_feature_set = feature_sets[best_accuracy_index]
    
#     print("\nAccuracies for each feature set:")
#     for i, (acc, feat_set) in enumerate(zip(accuracies, feature_sets)):
#         print(f"Features: {len(feat_set)}, Accuracy: {acc:.4f}")
#         if i == best_accuracy_index:
#             print("^ Best performing set")
    
#     return best_feature_set[::-1] + selected_features[::-1]

# # Generate example data
# np.random.seed(42)
# X = np.random.rand(100, 20)
# y = (X[:, 0] + X[:, 1] + X[:, 2] > 1.5).astype(int)  # Binary classification based on first 3 features

# selected_features = recursive_feature_elimination(X, y)
# print("\nSelected features (in order of importance):", selected_features)
# print(f"Optimal number of features: {len(selected_features)}")

In [9]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, Dropout, GlobalAveragePooling1D, Dense
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import confusion_matrix, roc_curve, auc
# import pandas as pd
# import os

# def normalize(data):
#   min = np.min(data,axis = 0)
#   max = np.max(data,axis = 0)
#   #print('min, max',min, max)
#   data =  (data-min)/(max-min)
#   return data

# def create_model(filter_sz=64, kernel_sz1=1, kernel_sz2=1, dropout=0.4, input_shape=[None, 1]):
#     model = Sequential()
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz1, strides=1, activation='relu', input_shape=input_shape, name='L1'))
#     model.add(Dropout(dropout))
#     model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz2, strides=1, activation='relu', name='L2'))
#     model.add(Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', name='L3'))
#     model.add(GlobalAveragePooling1D())
#     model.add(Dropout(dropout))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1, activation='sigmoid', name='classification'))
#     adam = tf.keras.optimizers.Adam(learning_rate=1.e-04)
#     model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
#     return model

# def cross_val_cumulative_accuracy(X, y, folds=5):
#     kf = StratifiedKFold(n_splits=folds, shuffle=False)
#     y_true, y_pred = np.array([]), np.array([])
    
#     for train_index, test_index in kf.split(X, y):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
        
#         X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#         X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
        
#         model = create_model(input_shape=[X_train.shape[1], 1])
#         model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=16, shuffle=True, verbose=0, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)])
        
#         y_pred_fold = model.predict(X_test).flatten()
#         y_true = np.append(y_true, y_test)
#         y_pred = np.append(y_pred, y_pred_fold)
    
#     fpr, tpr, thresholds = roc_curve(y_true, y_pred)
#     gmeans = np.sqrt(tpr * (1 - fpr))
#     ix = np.argmax(gmeans)
#     tn, fp, fn, tp = confusion_matrix(y_true, (y_pred >= thresholds[ix])).ravel()
#     accuracy = (tp + tn) / (tn + fp + fn + tp)
    
#     return accuracy

# def recursive_feature_elimination(X, y):
#     n_features = X.shape[1]
#     feature_indices = list(range(n_features))
#     selected_features = []
    
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     accuracies = []
#     feature_sets = []
    
#     while len(feature_indices) > 1:
#         current_accuracy = cross_val_cumulative_accuracy(X_scaled[:, feature_indices], y)
#         accuracies.append(current_accuracy)
#         feature_sets.append(feature_indices.copy())
        
#         print(f"Current number of features: {len(feature_indices)}, Accuracy: {current_accuracy:.4f}")
        
#         feature_importances = []
#         for i in feature_indices:
#             temp_features = [f for f in feature_indices if f != i]
#             accuracy = cross_val_cumulative_accuracy(X_scaled[:, temp_features], y)
#             feature_importances.append((i, accuracy))
        
#         worst_feature, _ = max(feature_importances, key=lambda x: x[1])
#         feature_indices.remove(worst_feature)
#         selected_features.append(worst_feature)
        
#         print(f"Removed feature {worst_feature}")
    
#     # Find the point where accuracy starts decreasing
#     best_accuracy_index = np.argmax(accuracies)
#     best_feature_set = feature_sets[best_accuracy_index]
    
#     print("\nAccuracies for each feature set:")
#     for i, (acc, feat_set) in enumerate(zip(accuracies, feature_sets)):
#         print(f"Features: {len(feat_set)}, Accuracy: {acc:.4f}")
#         if i == best_accuracy_index:
#             print("^ Best performing set")
    
#     return best_feature_set[::-1] + selected_features[::-1]

# # Generate example data
# # np.random.seed(42)
# # X = np.random.rand(100, 20)
# # y = (X[:, 0] + X[:, 1] + X[:, 2] > 1.5).astype(int)  # Binary classification based on first 3 features

# # Read data from local directory
# st_num = 0
# dir = r'C:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\EEG_fNIRS_paper_Brain_informatics\channelEEG_codes_results_alphaBand\Results\Connectivities_LSTMED_multiChanROI_Copy'
# filename = 'exp_nov_winSiz_5ST'+str(st_num+1)+'_Copy.csv'
# print(f'working on :{filename}')

# data = pd.read_csv(os.path.join(dir,filename)).values
# m,n = data.shape
# X = normalize(data[:,0:n-1])

# #print(X0.shape)
# #X = X0.reshape(X0.shape[0],X0.shape[1],1)
# # X = normalize(X)
# print(X.shape)

# #print(X.shape)
# y = data[:,n-1]
# y = np.array([0 if y[i]==-1 else 1 for i in range(len(y))])


# selected_features = recursive_feature_elimination(X, y)
# print("\nSelected features (in order of importance):", selected_features)
# print(f"Optimal number of features: {len(selected_features)}")

working on :exp_nov_winSiz_5ST1_Copy.csv
(68, 20)


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Current number of features: 20, Accuracy: 0.5588


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Current number of features: 2, Accuracy: 0.6029


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Removed feature 4

Accuracies for each feature set:
Features: 20, Accuracy: 0.5588
Features: 19, Accuracy: 0.5735
Features: 18, Accuracy: 0.5588
Features: 17, Accuracy: 0.5735
Features: 16, Accuracy: 0.5588
Features: 15, Accuracy: 0.6324
Features: 14, Accuracy: 0.5147
Features: 13, Accuracy: 0.6324
Features: 12, Accuracy: 0.6176
Features: 11, Accuracy: 0.6176
Features: 10, Accuracy: 0.6618
^ Best performing set
Features: 9, Accuracy: 0.6176
Features: 8, Accuracy: 0.5735
Features: 7, Accuracy: 0.5441
Features: 6, Accuracy: 0.5882
Features: 5, Accuracy: 0.5588
Features: 4, Accuracy: 0.6029
Features: 3, Accuracy: 0.5000
Features: 2, Accuracy: 0.6029

Selected features (in order of importance): [19, 18, 17, 15, 13, 11, 8, 7, 4, 1, 4, 19, 1, 11, 8, 7, 18, 15, 13, 3, 14, 16, 6, 2, 10, 12, 0, 5, 9]
Optima

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, GlobalAveragePooling1D, Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc
import os 
import pandas as pd
from openpyxl import Workbook


def normalize(data):
    min = np.min(data, axis=0)
    max = np.max(data, axis=0)
    data = (data - min) / (max - min)
    return data

def create_model(filter_sz=64, kernel_sz1=3, kernel_sz2=3, dropout=0.4, input_shape=[None, 1]):
    model = Sequential()
    model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz1, strides=1, activation='relu', input_shape=input_shape, name='L1'))
    model.add(Dropout(dropout))
    model.add(Conv1D(filters=filter_sz, kernel_size=kernel_sz2, strides=1, activation='relu', name='L2'))
    model.add(Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', name='L3'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid', name='classification'))
    adam = tf.keras.optimizers.Adam(learning_rate=1.e-04)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

def compute_metrics(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    area = auc(fpr, tpr)
    gmeans = np.sqrt(tpr * (1 - fpr))
    ix = np.argmax(gmeans)
    tn, fp, fn, tp = confusion_matrix(y_true, (y_pred >= thresholds[ix])).ravel()
    N = tn + fp + fn + tp
    S = (tp + fn) / N
    P = (tp + fp) / N
    MCC = ((tp / N) - S * P) / np.sqrt(P * S * (1 - S) * (1 - P))
    accuracy = (tp + tn) / N
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return accuracy, area, MCC, sensitivity, specificity

def cross_val_cumulative_accuracy(X, y, folds=5):
    kf = StratifiedKFold(n_splits=folds, shuffle=False)
    y_true, y_pred = np.array([]), np.array([])
    _, n = X.shape
    filter_size = 64

    if n > 12:
        kernel_sz1 = 5
        kernel_sz2 = 7
    elif n >= 5 and n <= 12: 
        kernel_sz1 = n - 2
        kernel_sz2 = 3
    else:
        kernel_sz1 = 1
        kernel_sz2 = 1

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
        
        model = create_model(filter_sz=filter_size, kernel_sz1=kernel_sz1, kernel_sz2=kernel_sz2, input_shape=[X_train.shape[1], 1])
        model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2000, batch_size=16, shuffle=True, verbose=0, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)])
        
        y_pred_fold = model.predict(X_test).flatten()
        y_true = np.append(y_true, y_test)
        y_pred = np.append(y_pred, y_pred_fold)
    
    return compute_metrics(y_true, y_pred)

def recursive_feature_elimination(X, y, min_features=1, connectivity_list=None, output_dir=None, st_num=None):
    n_features = X.shape[1]
    feature_indices = list(range(n_features))
    selected_features = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    metrics_list = []
    feature_sets = []
    
    # Create a new Excel workbook
    wb = Workbook()
    ws = wb.active
    ws.title = "Feature Sets"
    ws.append(["Set", "Accuracy", "AUC", "MCC", "Sensitivity", "Specificity"] + [f"Feature_{i}" for i in range(n_features)])
    
    set_counter = 1
    
    while len(feature_indices) >= min_features:
        accuracy, auc_score, mcc, sensitivity, specificity = cross_val_cumulative_accuracy(X_scaled[:, feature_indices], y)
        metrics_list.append((accuracy, auc_score, mcc, sensitivity, specificity))
        feature_sets.append(feature_indices.copy())
        
        print(f"\nCurrent number of features: {len(feature_indices)}")
        print(f"Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, MCC: {mcc:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")
        print("Current feature set:")
        for idx in feature_indices:
            if connectivity_list:
                print(f"  Index {idx}: {connectivity_list[idx]}")
            else:
                print(f"  Index {idx}")
        
        # Write to Excel
        excel_row = [set_counter, accuracy, auc_score, mcc, sensitivity, specificity] + [''] * n_features
        for idx in feature_indices:
            excel_row[idx + 6] = connectivity_list[idx] if connectivity_list else idx
        ws.append(excel_row)
        set_counter += 1
        
        if len(feature_indices) == min_features:
            break
        
        feature_importances = []
        for i in feature_indices:
            temp_features = [f for f in feature_indices if f != i] # Removes one features at at time and check the accuracy
            if temp_features:  # Only compute metrics if there's at least one feature left
                temp_metrics = cross_val_cumulative_accuracy(X_scaled[:, temp_features], y)
                feature_importances.append((i, temp_metrics[0]))  # Using accuracy for feature importance
        
        if feature_importances:
            worst_feature, _ = max(feature_importances, key=lambda x: x[1])
            feature_indices.remove(worst_feature)  # removes the feature that has least contribution/effect on the accuray
            selected_features.append(worst_feature)
            
            if connectivity_list:
                print(f"Removed feature {worst_feature}: {connectivity_list[worst_feature]}")
            else:
                print(f"Removed feature {worst_feature}")
        else:
            break  # Stop if we can't remove any more features
    
    # Find the point where accuracy is highest
    best_metrics_index = max(range(len(metrics_list)), key=lambda i: metrics_list[i][0])  # Using accuracy for best set
    best_feature_set = feature_sets[best_metrics_index]
    best_metrics = metrics_list[best_metrics_index]
    
    print("\nMetrics for each feature set:")
    for i, (metrics, feat_set) in enumerate(zip(metrics_list, feature_sets)):
        print(f"Features: {len(feat_set)}, Accuracy: {metrics[0]:.4f}, AUC: {metrics[1]:.4f}, MCC: {metrics[2]:.4f}, Sensitivity: {metrics[3]:.4f}, Specificity: {metrics[4]:.4f}")
        if i == best_metrics_index:
            print("^ Best performing set")
            print("Best feature set:")
            for idx in feat_set:
                if connectivity_list:
                    print(f"  Index {idx}: {connectivity_list[idx]}")
                else:
                    print(f"  Index {idx}")
    
    print("\n" + "="*50)
    print(f"Best Feature Set: {best_feature_set}")
    if connectivity_list:
        print("Best Feature Set Connectivities:")
        for idx in best_feature_set:
            print(f"  Index {idx}: {connectivity_list[idx]}")
    print(f"Best Metrics - Accuracy: {best_metrics[0]:.4f}, AUC: {best_metrics[1]:.4f}, MCC: {best_metrics[2]:.4f}, Sensitivity: {best_metrics[3]:.4f}, Specificity: {best_metrics[4]:.4f}")
    print("="*50)
    
    # Save Excel file
    if output_dir and st_num is not None:
        #excel_filename = os.path.join(output_dir, f'ST_{st_num}_feature_sets_metrics.xlsx')
        excel_filename = os.path.join(output_dir, f'P_{st_num}_feature_sets_metrics.xlsx')
        wb.save(excel_filename)
        print(f"Excel file saved: {excel_filename}")
    
    return best_feature_set, best_metrics

# Directory and file details
dir = r'C:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\EEG_fNIRS_paper_Brain_informatics\channelEEG_codes_results_alphaBand\Results\Connectivities_LSTMED_multiChanROI_Copy'
output_directory = r'C:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\EEG_fNIRS_paper_Brain_informatics\channelEEG_codes_results_alphaBand\Results\Connectivities_LSTMED_multiChanROI_Copy'

connectivity = ['LPFC-->RPFC', 'LPFC-->LPMC', 'LPFC-->RPMC', 'LPFC-->SMA', 'RPFC-->LPFC', 'RPFC-->LPMC', 'RPFC-->RPMC', 'RPFC-->SMA', 'LPMC-->LPFC', 'LPMC-->RPFC',
                'LPMC-->RPMC', 'LPMC-->SMA', 'RPMC-->LPFC', 'RPMC-->RPFC', 'RPMC-->LPMC', 'RPMC-->SMA', 'SMA-->LPFC', 'SMA-->RPFC', 'SMA-->LPMC', 'SMA-->RPMC']

# Loop over st_num
for st_num in range(1, 5):
    #filename = f'exp_nov_winSiz_5ST{st_num}_Copy.csv'
    filename = f'exp_nov_winSiz_5P{st_num}_Copy.csv'
    print(f'Working on: {filename}')
    data = pd.read_csv(os.path.join(dir, filename)).values
    m, n = data.shape
    X = normalize(data[:, 0:n-1])
    
    print(X.shape)
    y = data[:, n-1]
    y = np.array([0 if y[i] == -1 else 1 for i in range(len(y))])
    
    # When calling the function, pass the connectivity list, output directory, and st_num
    best_features, best_metrics = recursive_feature_elimination(X, y, min_features=1, connectivity_list=connectivity, output_dir=output_directory, st_num=st_num)
    print("\nSelected features (in order of importance):", best_features)
    print(f"Optimal number of features: {len(best_features)}")
    print(f"Best metrics - Accuracy: {best_metrics[0]:.4f}, AUC: {best_metrics[1]:.4f}, MCC: {best_metrics[2]:.4f}, Sensitivity: {best_metrics[3]:.4f}, Specificity: {best_metrics[4]:.4f}")


Working on: exp_nov_winSiz_5P1_Copy.csv
(62, 20)


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step

Current number of features: 20
Accuracy: 0.9032, AUC: 0.8763, MCC: 0.7932, Sensitivity: 0.7826, Specificity: 0.9744
Current feature set:
  Index 0: LPFC-->RPFC
  Index 1: LPFC-->LPMC
  Index 2: LPFC-->RPMC
  Index 3: LPFC-->SMA
  Index 4: RPFC-->LPFC
  Index 5: RPFC-->LPMC
  Index 6: RPFC-->RPMC
  Index 7: RPFC-->SMA
  Index 8: LPMC-->LPFC
  Index 9: LPMC-->RPFC
  Index 10: LPMC-->RPMC
  Index 11: LPMC-->SMA
  Index 12: RPMC-->LPFC
  Index 13: RPMC-->RPFC
  Index 14: RPMC-->LPMC
  Index 15: RPMC-->SMA
  Index 16: SMA-->LPFC
  Index 17: SMA-->RPFC
  Index 18: SMA-->LPMC
  Index 19: SMA-->RPMC


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Removed feature 1: LPFC-->LPMC


c:\Users\_Kamat_\Desktop\RPI\ResearchWork\Papers_\Effective_Connectivity\tfLSTM_ED\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step

Current number of features: 1
Accuracy: 0.7209, AUC: 0.7443, MCC: 0.4470, Sensitivity: 0.7647, Specificity: 0.6923
Current feature set:
  Index 10: LPMC-->RPMC

Metrics for each feature set:
Features: 20, Accuracy: 0.6047, AUC: 0.6380, MCC: 0.3651, Sensitivity: 0.9412, Specificity: 0.3846
Features: 19, Accuracy: 0.6279, AUC: 0.6380, MCC: 0.2772, Sensitivity: 0.7059, Specificity: 0.5769
Features: 18, Accuracy: 0.6047, AUC: 0.6041, MCC: 0.3252, Sensitivity: 0.8824, Specificity: 0.4231
Features: 17, Accuracy: 0.6512, AUC: 0.6403, MCC: 0.4287, Sensitivity: 0.9412, Specificity: 0.4615
Features: 16, Accuracy: 0.6279, AUC: 0.6312, MCC: 0.3005, Sensitivity: 0.7647, Specificity: 0.5385
Features: 15, Accuracy: 0.6279, AUC: 0.6222, MCC: 0.3970, Sensitivity: 0.9412, Specificity: 0.4231
Features: 14, Accuracy: 